In [95]:
import pandas as pd, numpy as np

df_geo_c = pd.read_csv("../data/cleaned/geo_2023.csv")
df_dg    = pd.read_csv("../data/cleaned/drug_geo_2023.csv")
df_sdh   = pd.read_csv("../data/cleaned/sdoh_2024.csv", header=0)

# Rebuild keys quickly
for df in [df_geo_c, df_dg]:
    if "prscrbr_geo_cd" in df.columns:
        df["fips"] = df["prscrbr_geo_cd"].astype(str).str.zfill(5)


/var/folders/3p/l7x9nvb10398yg6vv8kxpstr0000gn/T/ipykernel_67590/674620366.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dg    = pd.read_csv("../data/cleaned/drug_geo_2023.csv")


In [96]:
# Rebuild FIPS for geo and drug_geo
for df in [df_geo_c, df_dg]:
    if "prscrbr_geo_cd" in df.columns:
        df["fips"] = df["prscrbr_geo_cd"].astype(str).str.zfill(5)
    elif "fips" in df.columns:
        df["fips"] = df["fips"].astype(str).str.zfill(5)

# For SDOH
if "fips" in df_sdh.columns:
    df_sdh["fips"] = df_sdh["fips"].astype(str).str.zfill(5)


In [97]:
print("geo_2023 shape:", df_geo_c.shape)
print("drug_geo shape:", df_dg.shape)
print("sdoh shape:", df_sdh.shape)

print("geo_2023 columns:", df_geo_c.columns.tolist()[:10])
print("drug_geo columns:", df_dg.columns.tolist()[:10])
print("sdoh columns:", df_sdh.columns.tolist()[:10])

print("Unique FIPS geo:", df_geo_c["fips"].nunique())
print("Unique FIPS drug_geo:", df_dg["fips"].nunique())
print("Unique FIPS sdh:", df_sdh["fips"].nunique())


geo_2023 shape: (328890, 20)
drug_geo shape: (1233548, 24)
sdoh shape: (3201, 272)
geo_2023 columns: ['year', 'prscrbr_geo_lvl', 'prscrbr_geo_cd', 'prscrbr_geo_desc', 'ruca_cd', 'breakout_type', 'breakout', 'tot_prscrbrs', 'tot_opioid_prscrbrs', 'tot_opioid_clms']
drug_geo columns: ['prscrbr_geo_lvl', 'prscrbr_geo_cd', 'prscrbr_geo_desc', 'brnd_name', 'gnrc_name', 'tot_prscrbrs', 'tot_clms', 'tot_30day_fills', 'tot_drug_cst', 'tot_benes']
sdoh columns: ['fips', 'state', 'county', 'unreliable', 'deaths', 'years_of_potential_life_lost_rate', '95pct_ci___low', '95pct_ci___high', 'national_z_score', 'ypll_rate_(hispanic_(all_races))']
Unique FIPS geo: 24551
Unique FIPS drug_geo: 78
Unique FIPS sdh: 3201


In [98]:
# Geo
if "prscrbr_geo_cd" in df_geo_c.columns:
    df_geo_c["fips"] = df_geo_c["prscrbr_geo_cd"].astype(str).str.zfill(5)

# Drug-Geo
if "prscrbr_geo_cd" in df_dg.columns:
    df_dg["fips"] = df_dg["prscrbr_geo_cd"].astype(str).str.zfill(5)

# SDOH
if "fips" in df_sdh.columns:
    df_sdh["fips"] = df_sdh["fips"].astype(str).str.zfill(5)

# Check
print("Unique FIPS in geo_2023:", df_geo_c["fips"].nunique())
print("Unique FIPS in drug_geo:", df_dg["fips"].nunique())
print("Unique FIPS in sdoh:", df_sdh["fips"].nunique())


Unique FIPS in geo_2023: 24551
Unique FIPS in drug_geo: 78
Unique FIPS in sdoh: 3201


In [99]:
# GEO dataset (prescribing rates by geography)
geo_keep = ["fips","year","opioid_prscrbng_rate",
            "tot_clms","tot_opioid_clms",
            "tot_prscrbrs","tot_opioid_prscrbrs",
            "la_tot_opioid_clms","la_opioid_prscrbng_rate"]
df_geo_small = df_geo_c[geo_keep]

# Drug-Geo dataset (drug costs, claims, LIS/non-LIS)
drug_keep = ["fips","year","tot_prscrbrs","tot_clms","tot_30day_fills",
             "tot_drug_cst","tot_benes","lis_bene_cst_shr","nonlis_bene_cst_shr"]
df_dg_small = df_dg[drug_keep]

# SDOH dataset (social determinants of health)
sdh_keep = ["fips","state","county","population",
            "pct_uninsured","pct_unemployed","pct_children_in_poverty","income_ratio",
            "mental_health_provider_rate","primary_care_physicians_rate",
            "pct_adults_reporting_currently_smoking","pct_adults_with_obesity",
            "pct_excessive_drinking","pct_severe_housing_problems","injury_death_rate"]
df_sdh_small = df_sdh[sdh_keep]


In [100]:
print(panel["fips"].dtype, df_sdh["fips"].dtype)
print(panel["fips"].head())
print(df_sdh.head(10))

object object
0    01001
1    01003
2    01005
3    01007
4    01009
Name: fips, dtype: object
    fips    state    county unreliable   deaths  \
0  01000  Alabama       NaN        NaN  98140.0   
1  01001  Alabama   Autauga        NaN    942.0   
2  01003  Alabama   Baldwin        NaN   3789.0   
3  01005  Alabama   Barbour        NaN    579.0   
4  01007  Alabama      Bibb        NaN    501.0   
5  01009  Alabama    Blount        NaN   1239.0   
6  01011  Alabama   Bullock        NaN    229.0   
7  01013  Alabama    Butler        NaN    449.0   
8  01015  Alabama   Calhoun        NaN   2735.0   
9  01017  Alabama  Chambers        NaN    767.0   

   years_of_potential_life_lost_rate  95pct_ci___low  95pct_ci___high  \
0                       11415.734833    11307.652847     11523.816818   
1                        9407.948438     8508.880150     10307.016727   
2                        8981.575353     8514.700923      9448.449784   
3                       13138.848362    11474.73401

In [101]:
# Step 1: Merge drug + geo prescribing
panel = df_dg_small.merge(
    df_geo_small[["fips","year","opioid_prscrbng_rate"]],
    on=["fips","year"], how="left"
)

print("After merging drug+geo:", panel.shape)

# Step 2: Add SDOH (cross-sectional, 2024 data)
panel = panel.merge(df_sdh_small, on="fips", how="left")

print("After adding SDOH:", panel.shape)


After merging drug+geo: (1369020, 10)
After adding SDOH: (1369020, 24)


In [102]:
print(panel.columns.tolist())

['fips', 'year', 'tot_prscrbrs', 'tot_clms', 'tot_30day_fills', 'tot_drug_cst', 'tot_benes', 'lis_bene_cst_shr', 'nonlis_bene_cst_shr', 'opioid_prscrbng_rate', 'state', 'county', 'population', 'pct_uninsured', 'pct_unemployed', 'pct_children_in_poverty', 'income_ratio', 'mental_health_provider_rate', 'primary_care_physicians_rate', 'pct_adults_reporting_currently_smoking', 'pct_adults_with_obesity', 'pct_excessive_drinking', 'pct_severe_housing_problems', 'injury_death_rate']


In [ ]:
print("Population missing %:", panel["population"].isna().mean()*100)
print("opioid_claims_share missing %:", panel["opioid_claims_share"].isna().mean()*100)
print("tot_clms missing %:", panel["tot_clms"].isna().mean()*100)

In [103]:
# Take population from SDOH
pop_lookup = df_sdh[["fips","population"]].drop_duplicates()

# Merge without year restriction (applies to all years in panel)
panel = panel.drop(columns=["population"], errors="ignore")
panel = panel.merge(pop_lookup, on="fips", how="left")

# Recalculate claims per 1k
eps = 1e-9
panel["claims_per_1k"] = panel["tot_opioid_clms"] / (panel["population"]/1000 + eps)

print(panel[["claims_per_1k"]].describe())

import numpy as np
eps = 1e-9  # to avoid divide-by-zero

# Recreate opioid cost (if not directly available)
panel["opioid_cost"] = panel["tot_opioid_clms"].fillna(0) * panel["cost_per_claim"].fillna(0)

# Compute per-capita cost
panel["opioid_cost_per_capita"] = (
    panel["opioid_cost"] / (panel["population"].replace(0, np.nan) + eps)
)

# Check results
print(panel["opioid_cost_per_capita"].describe())
print("Non-missing count:", panel["opioid_cost_per_capita"].notna().sum())



KeyError: 'tot_opioid_clms'

In [104]:
print(df_sdh.columns.tolist())
print(df_sdh[["fips","population"]].head())

['fips', 'state', 'county', 'unreliable', 'deaths', 'years_of_potential_life_lost_rate', '95pct_ci___low', '95pct_ci___high', 'national_z_score', 'ypll_rate_(hispanic_(all_races))', 'ypll_rate_(hispanic_(all_races))_95pct_ci___low', 'ypll_rate_(hispanic_(all_races))_95pct_ci___high', 'ypll_rate_(hispanic_(all_races))_unreliable', 'ypll_rate_(non_hispanic_aian)', 'ypll_rate_(non_hispanic_aian)_95pct_ci___low', 'ypll_rate_(non_hispanic_aian)_95pct_ci___high', 'ypll_rate_(non_hispanic_aian)_unreliable', 'ypll_rate_(non_hispanic_asian)', 'ypll_rate_(non_hispanic_asian)_95pct_ci___low', 'ypll_rate_(non_hispanic_asian)_95pct_ci___high', 'ypll_rate_(non_hispanic_asian)_unreliable', 'ypll_rate_(non_hispanic_black)', 'ypll_rate_(non_hispanic_black)_95pct_ci___low', 'ypll_rate_(non_hispanic_black)_95pct_ci___high', 'ypll_rate_(non_hispanic_black)_unreliable', 'ypll_rate_(non_hispanic_native_hawaiian_and_other_pacific_islande ', 'ypll_rate_(non_hispanic_native_hawaiian_and_other_pacific_islande _

In [105]:
df_sdh["fips"] = df_sdh["fips"].astype(str).str.zfill(5)
panel["fips"] = panel["fips"].astype(str).str.zfill(5)

In [106]:
print("Sample panel FIPS:", panel["fips"].dropna().unique()[:10])
print("Sample SDOH FIPS:", df_sdh["fips"].dropna().unique()[:10])
overlap = set(panel["fips"]).intersection(set(df_sdh["fips"]))
print("Number of overlapping FIPS:", len(overlap))

Sample panel FIPS: ['00nan' '001.0' '002.0' '004.0' '005.0' '006.0' '008.0' '009.0' '010.0'
 '011.0']
Sample SDOH FIPS: ['01000' '01001' '01003' '01005' '01007' '01009' '01011' '01013' '01015'
 '01017']
Number of overlapping FIPS: 0


In [107]:
geo_2023 = pd.read_csv("../data/cms/OpioidPrescribingRates_Geography/2023/OMT_MDCR_RY25_P04_V10_Y23_GEO.csv")
print(geo_2023.shape)
print(geo_2023.columns.tolist())
geo_2023.head(10)

(328890, 18)
['Year', 'Prscrbr_Geo_Lvl', 'Prscrbr_Geo_Cd', 'Prscrbr_Geo_Desc', 'RUCA_Cd', 'Breakout_Type', 'Breakout', 'Tot_Prscrbrs', 'Tot_Opioid_Prscrbrs', 'Tot_Opioid_Clms', 'Tot_Clms', 'Opioid_Prscrbng_Rate', 'Opioid_Prscrbng_Rate_5Y_Chg', 'Opioid_Prscrbng_Rate_1Y_Chg', 'LA_Tot_Opioid_Clms', 'LA_Opioid_Prscrbng_Rate', 'LA_Opioid_Prscrbng_Rate_5Y_Chg', 'LA_Opioid_Prscrbng_Rate_1Y_Chg']


,Year,Prscrbr_Geo_Lvl,Prscrbr_Geo_Cd,Prscrbr_Geo_Desc,RUCA_Cd,Breakout_Type,Breakout,Tot_Prscrbrs,Tot_Opioid_Prscrbrs,Tot_Opioid_Clms,Tot_Clms,Opioid_Prscrbng_Rate,Opioid_Prscrbng_Rate_5Y_Chg,Opioid_Prscrbng_Rate_1Y_Chg,LA_Tot_Opioid_Clms,LA_Opioid_Prscrbng_Rate,LA_Opioid_Prscrbng_Rate_5Y_Chg,LA_Opioid_Prscrbng_Rate_1Y_Chg
0,2023,National,NaN,National,NaN,Totals,Overall,1379131.0,796960.0,59311234.0,1.614516e+09,3.67,-1.01,-0.21,5266348.0,8.88,-2.91,-0.49
1,2023,National,NaN,National,NaN,Rural/Urban,Rural,150022.0,96471.0,10433987.0,2.765332e+08,3.77,-1.04,-0.20,859739.0,8.24,-2.81,-0.47
2,2023,National,NaN,National,NaN,Rural/Urban,Urban,1228080.0,699913.0,48835671.0,1.337145e+09,3.65,-1.01,-0.21,5063859.0,10.37,-2.70,-0.46
3,2023,State,1.0,Alabama,NaN,Totals,Overall,17808.0,9741.0,1661292.0,3.058605e+07,5.43,-1.47,-0.26,101471.0,6.11,-2.25,-0.38
4,2023,State,2.0,Alaska,NaN,Totals,Overall,3034.0,1880.0,73416.0,1.812674e+06,4.05,-1.77,-0.34,9244.0,12.59,-6.04,-0.86
5,2023,State,4.0,Arizona,NaN,Totals,Overall,29350.0,16696.0,1213710.0,2.859759e+07,4.24,-1.38,-0.27,130736.0,10.77,-4.03,-0.57
6,2023,State,5.0,Arkansas,NaN,Totals,Overall,11820.0,7343.0,941497.0,1.917052e+07,4.91,-0.99,-0.20,54862.0,5.83,-2.73,-0.46
7,2023,State,6.0,California,NaN,Totals,Overall,139039.0,78516.0,5022505.0,1.458988e+08,3.44,-1.15,-0.26,437224.0,8.71,-2.55,-0.38
8,2023,State,8.0,Colorado,NaN,Totals,Overall,23898.0,15621.0,926893.0,1.922503e+07,4.82,-1.30,-0.23,113448.0,12.24,-1.96,-0.05
9,2023,State,9.0,Connecticut,NaN,Totals,Overall,19204.0,10418.0,525553.0,1.823245e+07,2.88,-0.68,-0.18,63419.0,12.07,-2.85,-0.51


In [56]:
!pip3 install us


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [108]:
import pandas as pd
import us  # pip install us

# Read raw file
geo_2023 = pd.read_csv(
    "../data/cms/OpioidPrescribingRates_Geography/2023/OMT_MDCR_RY25_P04_V10_Y23_GEO.csv",
    dtype=str  # force all columns to string at load
)

# Convert numerics back later if needed
numeric_cols = ["Tot_Clms","Tot_Opioid_Clms","Tot_Prscrbrs","Tot_Opioid_Prscrbrs",
                "Opioid_Prscrbng_Rate","LA_Opioid_Prscrbng_Rate","Year"]
for c in numeric_cols:
    if c in geo_2023.columns:
        geo_2023[c] = pd.to_numeric(geo_2023[c], errors="coerce")

# Ensure these two are always strings
geo_2023["Prscrbr_Geo_Lvl"] = geo_2023["Prscrbr_Geo_Lvl"].fillna("").astype(str)
geo_2023["Prscrbr_Geo_Desc"] = geo_2023["Prscrbr_Geo_Desc"].fillna("").astype(str)

# Keep only county-level rows
geo_county = geo_2023[geo_2023["Prscrbr_Geo_Lvl"].str.lower() == "county"].copy()

# Split "County, ST" into two parts
geo_county["county_name"] = geo_county["Prscrbr_Geo_Desc"].apply(
    lambda x: x.split(",")[0].strip() if "," in x else None
)
geo_county["state_abbr"] = geo_county["Prscrbr_Geo_Desc"].apply(
    lambda x: x.split(",")[1].strip() if "," in x else None
)

# Map state abbreviation → state FIPS
state_fips_map = {s.abbr: s.fips for s in us.states.STATES}
geo_county["state_fips"] = geo_county["state_abbr"].map(state_fips_map)

# County FIPS
geo_county["fips"] = geo_county["Prscrbr_Geo_Cd"].astype(str).str.zfill(5)

# Rename for consistency
geo_county = geo_county.rename(columns={
    "Year": "year",
    "Tot_Clms": "tot_clms",
    "Tot_Opioid_Clms": "tot_opioid_clms",
    "Tot_Prscrbrs": "tot_prscrbrs",
    "Tot_Opioid_Prscrbrs": "tot_opioid_prscrbrs",
    "Opioid_Prscrbng_Rate": "opioid_prscrbng_rate",
    "LA_Opioid_Prscrbng_Rate": "la_opioid_prscrbng_rate"
})

print(geo_county[["fips","county_name","state_abbr","year"]].head())
print("Unique FIPS count:", geo_county["fips"].nunique())


      fips county_name state_abbr  year
171  01001        None       None  2023
172  01003        None       None  2023
173  01005        None       None  2023
174  01007        None       None  2023
175  01009        None       None  2023
Unique FIPS count: 3202


In [70]:
# Load SDOH (already extracted earlier)
df_sdh = pd.read_csv(
    "../data/cleaned/sdoh_2024.csv",
   header=0
)

# Standardize FIPS: always 5-digit string
df_sdh["fips"] = df_sdh["fips"].astype(str).str.zfill(5)

print("SDOH FIPS sample:", df_sdh["fips"].head())
print("Overlap with geo_county:", len(set(df_sdh["fips"]) & set(geo_county["fips"])))


SDOH FIPS sample: 0    01000
1    01001
2    01003
3    01005
4    01007
Name: fips, dtype: object
Overlap with geo_county: 3107


In [109]:
print(df_sdh.columns.tolist()[:20])  # show first 20 col names

['fips', 'state', 'county', 'unreliable', 'deaths', 'years_of_potential_life_lost_rate', '95pct_ci___low', '95pct_ci___high', 'national_z_score', 'ypll_rate_(hispanic_(all_races))', 'ypll_rate_(hispanic_(all_races))_95pct_ci___low', 'ypll_rate_(hispanic_(all_races))_95pct_ci___high', 'ypll_rate_(hispanic_(all_races))_unreliable', 'ypll_rate_(non_hispanic_aian)', 'ypll_rate_(non_hispanic_aian)_95pct_ci___low', 'ypll_rate_(non_hispanic_aian)_95pct_ci___high', 'ypll_rate_(non_hispanic_aian)_unreliable', 'ypll_rate_(non_hispanic_asian)', 'ypll_rate_(non_hispanic_asian)_95pct_ci___low', 'ypll_rate_(non_hispanic_asian)_95pct_ci___high']


In [110]:
df_sdh["fips"] = df_sdh["fips"].astype(str).str.split(".").str[0].str.zfill(5)

In [111]:
for df in [df_geo_c, df_prescribers, df_dg]:
    if "prscrbr_geo_cd" in df.columns:
        df["fips"] = df["prscrbr_geo_cd"].astype(str).str.split(".").str[0].str.zfill(5)

In [112]:
print("Sample FIPS in CMS:", df_geo_c["fips"].dropna().unique()[:10])
print("Sample FIPS in SDOH:", df_sdh["fips"].dropna().unique()[:10])

Sample FIPS in CMS: ['00nan' '00001' '00002' '00004' '00005' '00006' '00008' '00009' '00010'
 '00011']
Sample FIPS in SDOH: ['01000' '01001' '01003' '01005' '01007' '01009' '01011' '01013' '01015'
 '01017']


In [113]:
overlap = set(df_geo_c["fips"]).intersection(set(df_sdh["fips"]))
print("Number of overlapping FIPS:", len(overlap))

Number of overlapping FIPS: 3124


In [114]:
print(panel.columns.tolist())

['fips', 'year', 'tot_prscrbrs', 'tot_clms', 'tot_30day_fills', 'tot_drug_cst', 'tot_benes', 'lis_bene_cst_shr', 'nonlis_bene_cst_shr', 'opioid_prscrbng_rate', 'state', 'county', 'pct_uninsured', 'pct_unemployed', 'pct_children_in_poverty', 'income_ratio', 'mental_health_provider_rate', 'primary_care_physicians_rate', 'pct_adults_reporting_currently_smoking', 'pct_adults_with_obesity', 'pct_excessive_drinking', 'pct_severe_housing_problems', 'injury_death_rate', 'population']


In [115]:
# Fix malformed FIPS in panel
import numpy as np

# 1. Keep only the useful columns from df_geo
geo_keep = [
    "fips", "year", "tot_clms", "tot_opioid_clms",
    "tot_prscrbrs", "tot_opioid_prscrbrs", "opioid_prscrbng_rate"
]
df_geo_small = df_geo_c[geo_keep].copy()

# 2. Select SDOH features
sdh_keep = [
    "fips","state","county","population",
    "pct_uninsured","pct_unemployed","pct_children_in_poverty","income_ratio",
    "mental_health_provider_rate","primary_care_physicians_rate",
    "pct_adults_reporting_currently_smoking","pct_adults_with_obesity",
    "pct_excessive_drinking","pct_severe_housing_problems","injury_death_rate"
]
df_sdh_small = df_sdh[sdh_keep].copy()

# 3. Merge CMS + SDOH
panel = df_geo_small.merge(df_sdh_small, on="fips", how="left")

# 4. Feature engineering
eps = 1e-9
panel["opioid_claims_share"] = panel["tot_opioid_clms"] / (panel["tot_clms"] + eps)
panel["cost_per_claim"] = panel["tot_opioid_clms"] / (panel["tot_prscrbrs"] + eps)

if "population" in panel.columns:
    panel["claims_per_1k"] = panel["tot_opioid_clms"] / (panel["population"]/1000 + eps)

# Log transforms to reduce skew
for c in ["Tot_Clms","Tot_Opioid_Clms","claims_per_1k"]:
    if c in panel.columns:
        panel[f"log1p_{c}"] = np.log1p(panel[c])

print("Final panel shape:", panel.shape)
print(panel.head())

Final panel shape: (328890, 25)
    fips  year      tot_clms  tot_opioid_clms  tot_prscrbrs  \
0  00nan  2023  1.614516e+09       59311234.0     1379131.0   
1  00nan  2023  2.765332e+08       10433987.0      150022.0   
2  00nan  2023  1.337145e+09       48835671.0     1228080.0   
3  00001  2023  3.058605e+07        1661292.0       17808.0   
4  00002  2023  1.812674e+06          73416.0        3034.0   

   tot_opioid_prscrbrs  opioid_prscrbng_rate state county  population  ...  \
0             796960.0                  3.67   NaN    NaN         NaN  ...   
1              96471.0                  3.77   NaN    NaN         NaN  ...   
2             699913.0                  3.65   NaN    NaN         NaN  ...   
3               9741.0                  5.43   NaN    NaN         NaN  ...   
4               1880.0                  4.05   NaN    NaN         NaN  ...   

   primary_care_physicians_rate  pct_adults_reporting_currently_smoking  \
0                           NaN              

In [116]:
panel = panel.drop(columns=["population"], errors="ignore")
panel = panel.merge(df_sdh[["fips","population"]], on="fips", how="left")

print("Population missing % after merge:", panel["population"].isna().mean()*100)

Population missing % after merge: 65.68609565508224


In [121]:
eps = 1e-9  # to avoid divide-by-zero

# Recreate opioid cost (if not directly available)
panel["opioid_cost"] = panel["tot_opioid_clms"].fillna(0) * panel["cost_per_claim"].fillna(0)

# Compute per-capita cost
panel["opioid_cost_per_capita"] = (
    panel["opioid_cost"] / (panel["population"].replace(0, np.nan) + eps)
)

# Check results
print(panel["opioid_cost_per_capita"].describe())
print("Non-missing count:", panel["opioid_cost_per_capita"].notna().sum())


count    112855.000000
mean         32.887651
std         107.745060
min           0.000000
25%           0.000000
50%           9.652977
75%          33.831463
max        6490.470000
Name: opioid_cost_per_capita, dtype: float64
Non-missing count: 112855


In [122]:
print(panel.columns.tolist())

['fips', 'year', 'tot_clms', 'tot_opioid_clms', 'tot_prscrbrs', 'tot_opioid_prscrbrs', 'opioid_prscrbng_rate', 'state', 'county', 'pct_uninsured', 'pct_unemployed', 'pct_children_in_poverty', 'income_ratio', 'mental_health_provider_rate', 'primary_care_physicians_rate', 'pct_adults_reporting_currently_smoking', 'pct_adults_with_obesity', 'pct_excessive_drinking', 'pct_severe_housing_problems', 'injury_death_rate', 'opioid_claims_share', 'cost_per_claim', 'claims_per_1k', 'log1p_claims_per_1k', 'population', 'opioid_cost', 'opioid_cost_per_capita']


In [123]:
#Save engineered dataset
out_path = "../data/cleaned/feature_engineered_panel.csv"
panel.to_csv(out_path, index=False)
print(f"Saved feature engineered dataset: {out_path}, shape: {panel.shape}")

Saved feature engineered dataset: ../data/cleaned/feature_engineered_panel.csv, shape: (328890, 27)


In [117]:
#Save engineered dataset
out_path = "../data/cleaned/feature_engineered_panel.csv"
panel.to_csv(out_path, index=False)
print(f"Saved feature engineered dataset: {out_path}, shape: {panel.shape}")

Saved feature engineered dataset: ../data/cleaned/feature_engineered_panel.csv, shape: (328890, 25)


In [124]:
#Quick sanity checks
print(panel.head())
print(panel.describe(include="all").T)

    fips  year      tot_clms  tot_opioid_clms  tot_prscrbrs  \
0  00nan  2023  1.614516e+09       59311234.0     1379131.0   
1  00nan  2023  2.765332e+08       10433987.0      150022.0   
2  00nan  2023  1.337145e+09       48835671.0     1228080.0   
3  00001  2023  3.058605e+07        1661292.0       17808.0   
4  00002  2023  1.812674e+06          73416.0        3034.0   

   tot_opioid_prscrbrs  opioid_prscrbng_rate state county  pct_uninsured  ...  \
0             796960.0                  3.67   NaN    NaN            NaN  ...   
1              96471.0                  3.77   NaN    NaN            NaN  ...   
2             699913.0                  3.65   NaN    NaN            NaN  ...   
3               9741.0                  5.43   NaN    NaN            NaN  ...   
4               1880.0                  4.05   NaN    NaN            NaN  ...   

   pct_excessive_drinking  pct_severe_housing_problems  injury_death_rate  \
0                     NaN                          NaN   